# Store Sales - Time Series Forecasting

### Use machine learning to predict grocery sales

Kaggle link: [Store Sales - Time Series Forecasting](https://www.kaggle.com/competitions/store-sales-time-series-forecasting)

### Context
Forecasts aren’t just for meteorologists. Governments forecast economic growth. Scientists attempt to predict the future population. And businesses forecast product demand—a common task of professional data scientists. Forecasts are especially relevant to brick-and-mortar grocery stores, which must dance delicately with how much inventory to buy. Predict a little over, and grocers are stuck with overstocked, perishable goods. Guess a little under, and popular items quickly sell out, leading to lost revenue and upset customers. More accurate forecasting, thanks to machine learning, could help ensure retailers please customers by having just enough of the right products at the right time.

Current subjective forecasting methods for retail have little data to back them up and are unlikely to be automated. The problem becomes even more complex as retailers add new locations with unique needs, new products, ever-transitioning seasonal tastes, and unpredictable product marketing.

### Potential Impact
If successful, you'll have flexed some new skills in a real world example. For grocery stores, more accurate forecasting can decrease food waste related to overstocking and improve customer satisfaction. The results of this ongoing competition, over time, might even ensure your local store has exactly what you need the next time you shop.

### Dataset Description
In this competition, you will predict sales for the thousands of product families sold at Favorita stores located in Ecuador. The training data includes dates, store and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models.

### File Descriptions and Data Field Information

**train.csv**
The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
**store_nbr** identifies the store at which the products are sold.
**family** identifies the type of product sold.
**sales** gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
**onpromotion** gives the total number of items in a product family that were being promoted at a store at a given date.

**test.csv**
The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
The dates in the test data are for the 15 days after the last date in the training data.

**sample_submission.csv**
A sample submission file in the correct format.

**stores.csv**
Store metadata, including city, state, type, and cluster.
cluster is a grouping of similar stores.

**oil.csv**
Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

**holidays_events.csv**
* Holidays and Events, with metadata
* NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
* Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

### Additional Notes

* Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
* A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

### Importing all libraries

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import *
from sklearn.linear_model import *
from math import *
from sklearn.ensemble import *
from sklearn.feature_selection import *
from sklearn.feature_extraction import *
from sklearn.naive_bayes import *
from sklearn.discriminant_analysis import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from sklearn.neighbors import *
from sklearn.cluster import *
from sklearn.svm import *
import warnings
warnings.filterwarnings("ignore")

### Importing the training datasets

In [19]:
df_train_1 = pd.read_csv("train1.csv")
df_train_1.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,1/1/13,1,AUTOMOTIVE,0.0,0
1,1,1/1/13,1,BABY CARE,0.0,0
2,2,1/1/13,1,BEAUTY,0.0,0
3,3,1/1/13,1,BEVERAGES,0.0,0
4,4,1/1/13,1,BOOKS,0.0,0


In [20]:
df_train_2 = pd.read_csv("train2.csv")
df_train_2.head()

,id,date,store_nbr,family,sales,onpromotion
0,262143,5/28/13,14,MEATS,310.173000,0
1,262144,5/28/13,14,PERSONAL CARE,227.000000,0
2,262145,5/28/13,14,PET SUPPLIES,0.000000,0
3,262146,5/28/13,14,PLAYERS AND ELECTRONICS,0.000000,0
4,262147,5/28/13,14,POULTRY,47.809002,0


In [21]:
df_train_3 = pd.read_csv("train3.csv")
df_train_3.head()

,id,date,store_nbr,family,sales,onpromotion
0,524286,10/22/13,2,HOME AND KITCHEN I,0.0,0
1,524287,10/22/13,2,HOME AND KITCHEN II,0.0,0
2,524288,10/22/13,2,HOME APPLIANCES,0.0,0
3,524289,10/22/13,2,HOME CARE,0.0,0
4,524290,10/22/13,2,LADIESWEAR,0.0,0


In [22]:
df_train_4 = pd.read_csv("train4.csv")
df_train_4.head()

,id,date,store_nbr,family,sales,onpromotion
0,786429,3/19/14,25,CELEBRATION,8.000,0
1,786430,3/19/14,25,CLEANING,485.000,0
2,786431,3/19/14,25,DAIRY,588.000,0
3,786432,3/19/14,25,DELI,116.357,0
4,786433,3/19/14,25,EGGS,119.000,0


In [23]:
print("Total length of training dataframes : ",(len(df_train_1)+len(df_train_2)+len(df_train_3)+len(df_train_4)))

Total length of training dataframes :  1048575


### Merging all training dataframes into one single dataframe

In [24]:
a = df_train_1.append(df_train_2)
b = df_train_3.append(df_train_4)
df_train = a.append(b)
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,1/1/13,1,AUTOMOTIVE,0.0,0
1,1,1/1/13,1,BABY CARE,0.0,0
2,2,1/1/13,1,BEAUTY,0.0,0
3,3,1/1/13,1,BEVERAGES,0.0,0
4,4,1/1/13,1,BOOKS,0.0,0


In [25]:
print("Total length of merged dataframe : ",(len(df_train)))

Total length of merged dataframe :  1048575


### Importing testing dataset

In [26]:
df_test = pd.read_csv("test.csv")
df_test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [27]:
print("Total length of testing dataframe : ",(len(df_test)))

Total length of testing dataframe :  28512
